# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import datetime
from scipy import stats #The SciPy stats module

In [2]:
import api_token  # handle iexcloud API tokens
BASE_URL, IEX_CLOUD_API_TOKEN = api_token.get_base_url(), api_token.get_api_token()

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': ' SSESBRNLATEAGTOQ CLKNLM EGA/)DA(',
 'calculationPrice': 'close',
 'open': 121.35,
 'openTime': 1637167236900,
 'openSource': 'filficoa',
 'close': 126.62,
 'closeTime': 1686132166408,
 'closeSource': 'iolcaffi',
 'high': 126.59,
 'highTime': 1661255434043,
 'highSource': 'ldrm y epdcneia1t5ue ei',
 'low': 123.74,
 'lowTime': 1684148925226,
 'lowSource': 'eu e ediaide nrlmptcy15',
 'latestPrice': 126.77,
 'latestSource': 'Close',
 'latestTime': 'December 10, 2020',
 'latestUpdate': 1611474831499,
 'latestVolume': 83857943,
 'iexRealtimePrice': 125.59,
 'iexRealtimeSize': 14,
 'iexLastUpdated': 1685036540434,
 'delayedPrice': 128,
 'delayedPriceTime': 1680897751704,
 'oddLotDelayedPrice': 127.3,
 'oddLotDelayedPriceTime': 1645748526942,
 'extendedPrice': 126.91,
 'extendedChange': -0.07,
 'extendedChangePercent': -0.00057,
 'extendedPriceTime': 1643673800766,
 'previousClose': 123.44,
 'previousVolume': 118032526,
 'ch

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
pe_ratio = data['peRatio']
pe_ratio

39.17

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.68,55.95,N/A
1,AAL,18.54,-1.32,N/A
2,AAP,164.81,23.53,N/A
3,AAPL,124.04,38.86,N/A
4,ABBV,112.60,23.83,N/A
...,...,...,...,...
500,YUM,110.04,33.06,N/A
501,ZBH,153.27,-201.56,N/A
502,ZBRA,388.46,42.83,N/A
503,ZION,43.50,18.48,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,34.77,2.06,575
1,NLOK,19.97,3.89,1001
2,BIO,575.41,4.78,34
3,UNM,24.97,5.1,800
4,KIM,15.05,6.77,1328
5,ALL,107.55,7.71,185
6,EBAY,51.20,7.73,390
7,BIIB,253.77,8,78
8,MET,47.62,8.1,419
9,PNC,144.36,8.76,138


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [13]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,212.840,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,232.430,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.78,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.120,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.780,N/A,14.32,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,55.600,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.742,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,136.430,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,10.240,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1782.020,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.867327
1      0.140594
2      0.455446
3      0.724752
4      0.453465
         ...   
500    0.669307
501    0.019802
502    0.772277
503    0.372277
504     0.79604
Name: PE Percentile, Length: 505, dtype: object
0       0.723762
1      0.0455446
2       0.413861
3       0.962376
4       0.859406
         ...    
500    0.0415842
501     0.378218
502     0.830693
503     0.114851
504     0.944554
Name: PB Percentile, Length: 505, dtype: object
0        0.80297
1      0.0316832
2       0.178218
3       0.835644
4       0.693069
         ...    
500     0.764356
501     0.621782
502     0.671287
503     0.406931
504     0.938614
Name: PS Percentile, Length: 505, dtype: object
0       0.857426
1      0.0277228
2       0.241584
3            0.8
4       0.512871
         ...    
500     0.738614
501     0.780198
502     0.825743
503     0.231683
504     0.851485
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.807921
1      0.0534653
2       0.120792
3       0.944

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.52,N/A,55.99,0.867327,7.4000,0.723762,6.9400,0.80297,33.454608,0.857426,13.070798,0.807921,N/A
1,AAL,18.70,N/A,-1.31,0.140594,-1.9200,0.0455446,0.4296,0.0316832,-6.189801,0.0277228,1.402951,0.0534653,N/A
2,AAP,162.49,N/A,24.02,0.455446,2.9500,0.413861,1.1300,0.178218,10.523658,0.241584,2.402270,0.120792,N/A
3,AAPL,128.48,N/A,39.10,0.724752,32.9900,0.962376,7.6500,0.835644,27.709068,0.8,20.373957,0.944554,N/A
4,ABBV,112.12,N/A,23.87,0.453465,12.5900,0.859406,4.8200,0.693069,16.497886,0.512871,9.618071,0.653465,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.30,N/A,33.39,0.669307,-4.0000,0.0415842,5.9800,0.764356,23.999201,0.738614,10.481796,0.69505,N/A
501,ZBH,148.31,N/A,-205.86,0.019802,2.6900,0.378218,4.4500,0.621782,25.934131,0.780198,7.471197,0.481188,N/A
502,ZBRA,383.87,N/A,42.88,0.772277,10.4500,0.830693,4.6400,0.671287,29.364069,0.825743,11.118795,0.720792,N/A
503,ZION,42.76,N/A,18.97,0.372277,1.0326,0.114851,2.4300,0.406931,10.382415,0.231683,2.678400,0.134653,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.52,N/A,55.99,0.867327,7.4000,0.723762,6.9400,0.80297,33.454608,0.857426,13.070798,0.807921,0.811881
1,AAL,18.70,N/A,-1.31,0.140594,-1.9200,0.0455446,0.4296,0.0316832,-6.189801,0.0277228,1.402951,0.0534653,0.059802
2,AAP,162.49,N/A,24.02,0.455446,2.9500,0.413861,1.1300,0.178218,10.523658,0.241584,2.402270,0.120792,0.28198
3,AAPL,128.48,N/A,39.10,0.724752,32.9900,0.962376,7.6500,0.835644,27.709068,0.8,20.373957,0.944554,0.853465
4,ABBV,112.12,N/A,23.87,0.453465,12.5900,0.859406,4.8200,0.693069,16.497886,0.512871,9.618071,0.653465,0.634455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.30,N/A,33.39,0.669307,-4.0000,0.0415842,5.9800,0.764356,23.999201,0.738614,10.481796,0.69505,0.581782
501,ZBH,148.31,N/A,-205.86,0.019802,2.6900,0.378218,4.4500,0.621782,25.934131,0.780198,7.471197,0.481188,0.456238
502,ZBRA,383.87,N/A,42.88,0.772277,10.4500,0.830693,4.6400,0.671287,29.364069,0.825743,11.118795,0.720792,0.764158
503,ZION,42.76,N/A,18.97,0.372277,1.0326,0.114851,2.4300,0.406931,10.382415,0.231683,2.678400,0.134653,0.252079


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

Enter the value of your portfolio:1000000


In [21]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/home/wengong/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,24.56,814,5.20,0.156436,0.4439,0.0475248,0.4102,0.029703,2.629579,0.0336634,0.397810,0.0039604,0.0542574
1,AIG,40.91,488,-7.00,0.10495,0.5539,0.0554455,0.7851,0.0772277,4.651196,0.0435644,0.784322,0.0178218,0.059802
2,AAL,18.70,1069,-1.31,0.140594,-1.9200,0.0455446,0.4296,0.0316832,-6.189801,0.0277228,1.402951,0.0534653,0.059802
3,MET,48.04,416,7.90,0.164356,0.5925,0.0574257,0.6533,0.0574257,4.575576,0.039604,0.661973,0.0118812,0.0661386
4,HFC,28.93,691,-29.86,0.0653465,0.8574,0.0811881,0.3137,0.0138614,6.279520,0.0633663,2.287867,0.116832,0.0681188
5,F,9.45,2116,-17.39,0.0811881,1.2000,0.135644,0.2915,0.0118812,5.027322,0.0475248,2.204056,0.112871,0.0778218
6,ALL,108.73,183,7.55,0.160396,1.3100,0.161386,0.7204,0.0633663,2.357432,0.0316832,0.693496,0.0138614,0.0861386
7,HIG,47.60,420,10.30,0.192079,0.9873,0.0990099,0.8550,0.0891089,4.158446,0.0376238,0.822932,0.019802,0.0875248
8,BA,244.15,81,-29.80,0.0673267,-11.1500,0.029703,2.2400,0.380198,-30.883590,0.0138614,-216.079226,0.0019802,0.0986139
9,KSS,40.34,495,-59.28,0.049505,1.3400,0.170297,0.3624,0.0217822,9.955562,0.213861,1.458689,0.0633663,0.103762


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
today = datetime.datetime.now()
xlsx_file = f"003-value_strategy_{today.strftime('%Y-%m-%d')}.xlsx"

In [22]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [24]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [25]:
writer.save()